In [5]:
import os
import pandas as pd 

In [6]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Development\\Python\\TGSRTC_Productivity\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Development\\Python\\TGSRTC_Productivity'

In [9]:
data = pd.read_csv("artifacts/data_ingestion/TGSRTC_Productivity.csv")
data.head()

,employee_id,age,depot,absent_days,tot_opd_kms,hours,tot_schedules,palle_schedules,suplux_schedules,express_schedules,...,bilirubin_value,bilirubin_interpret,total_cholestrol,final_grading,PERF,ECG_interpret,BMI,alcohol,smoke,tobacco
0,152463,46,Mahaboobnagar,5,82432,3570,188.0,186.0,1,1.0,...,0.95,Normal,148,D,650.0,Within Normal Limits,23.0,0,0,0
1,152461,54,Mahaboobnagar,21,109731,3440,192.0,3.0,174,15.0,...,0.67,Normal,193,A,550.0,Within Normal Limits,25.0,0,0,1
2,152307,48,Mahaboobnagar,5,104597,3267,297.0,96.0,0,201.0,...,0.48,Normal,158,C,NaN,Intraventricular Conduction Defect? Lbbb Witho...,NaN,0,0,0
3,150906,54,Mahaboobnagar,0,3547,121,11.0,5.0,0,6.0,...,0.89,Normal,131,C,225.0,Intraventricular Conduction Defect? Lbbb Witho...,25.0,0,0,0
4,152071,51,Mahaboobnagar,5,76170,3832,214.5,210.5,0,4.0,...,0.88,Normal,128,C,350.0,Within Normal Limits,20.0,0,0,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   employee_id               511 non-null    int64  
 1   age                       511 non-null    int64  
 2   depot                     511 non-null    object 
 3   absent_days               511 non-null    int64  
 4   tot_opd_kms               511 non-null    int64  
 5   hours                     511 non-null    int64  
 6   tot_schedules             511 non-null    float64
 7   palle_schedules           511 non-null    float64
 8   suplux_schedules          511 non-null    int64  
 9   express_schedules         511 non-null    float64
 10  day_schedules             511 non-null    float64
 11  night_schedules           511 non-null    float64
 12  bus_age_0to5              511 non-null    int64  
 13  bus_age_5to10             511 non-null    float64
 14  bus_age_ab

In [11]:
data.isnull().sum()

employee_id                   0
age                           0
depot                         0
absent_days                   0
tot_opd_kms                   0
hours                         0
tot_schedules                 0
palle_schedules               0
suplux_schedules              0
express_schedules             0
day_schedules                 0
night_schedules               0
bus_age_0to5                  0
bus_age_5to10                 0
bus_age_above10               0
bus_kms_0to5l                 0
bus_kms_5to10l                0
bus_kms_above10l              0
creatinine_value              0
creatinine_interpret          0
blood_pressure_systolic       0
blood_pressure_diastolic      0
blood_pressure_interpret      0
hemoglobin_value              0
hemoglobin_interpret          0
glucose_random_value          0
glucose_interpret             0
bilirubin_value               0
bilirubin_interpret           0
total_cholestrol              0
final_grading                 0
PERF    

In [12]:
data.shape

(511, 37)

In [13]:
data.columns

Index(['employee_id', 'age', 'depot', 'absent_days', 'tot_opd_kms', 'hours',
       'tot_schedules', 'palle_schedules', 'suplux_schedules',
       'express_schedules', 'day_schedules', 'night_schedules', 'bus_age_0to5',
       'bus_age_5to10', 'bus_age_above10', 'bus_kms_0to5l', 'bus_kms_5to10l',
       'bus_kms_above10l', 'creatinine_value', 'creatinine_interpret',
       'blood_pressure_systolic', 'blood_pressure_diastolic',
       'blood_pressure_interpret', 'hemoglobin_value', 'hemoglobin_interpret',
       'glucose_random_value', 'glucose_interpret', 'bilirubin_value',
       'bilirubin_interpret', 'total_cholestrol', 'final_grading', 'PERF',
       'ECG_interpret', 'BMI', 'alcohol', 'smoke', 'tobacco'],
      dtype='object')

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [15]:
from TGSRTC_Productivity.constants import *
from TGSRTC_Productivity.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config


In [16]:
import os
from TGSRTC_Productivity import logger

In [23]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            print("Data Directory ***:", self.config.unzip_data_dir)
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            print("All columns:", all_cols)
            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [25]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-09-24 13:19:17,310: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-24 13:19:17,322: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-24 13:19:17,333: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-24 13:19:17,339: INFO: common: created directory at: artifacts]
[2024-09-24 13:19:17,343: INFO: common: created directory at: artifacts/data_validation]
All columns: ['employee_id', 'age', 'depot', 'absent_days', 'tot_opd_kms', 'hours', 'tot_schedules', 'palle_schedules', 'suplux_schedules', 'express_schedules', 'day_schedules', 'night_schedules', 'bus_age_0to5', 'bus_age_5to10', 'bus_age_above10', 'bus_kms_0to5l', 'bus_kms_5to10l', 'bus_kms_above10l', 'creatinine_value', 'creatinine_interpret', 'blood_pressure_systolic', 'blood_pressure_diastolic', 'blood_pressure_interpret', 'hemoglobin_value', 'hemoglobin_interpret', 'glucose_random_value', 'glucose_interpret', 'bilirubin_value', 'bilirubin_interpret', 'total_cholestrol